In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt
from tqdm import tqdm
from tqdm import trange
from skimage.transform import radon
from skimage.transform import iradon
from skimage.transform import rescale

import sys
sys.path.append('/Users/46h/Research/')
from accphys.tools import utils
from accphys.tools import plotting as myplt

In [ ]:
pplt.rc['grid.alpha'] = 0.05
pplt.rc['axes.grid'] = False
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['savefig.transparent'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap'] = 'viridis'

# Tomographic reconstruction in four dimensions 

Tomographic reconstruction — the reconstruction of an image from projections — is a well-developed area of image processing. The most well-known application is in medical imaging, in which the goal is to reconstruct a 3D image of the human body.

An image can be thought of as a *distribution* $f(\mathbf{x})$ which gives the density at point $\mathbf{x}$. A projection of a distribution is a sum along some direction; for example, a 1D projection $f(x)$ of the 2D distribution $f(x, y)$ is

$$
f(x) = \int_{-\infty}^{\infty}{f(x, y)dy}.
$$

When modeling a beam in accelerator, it's important to know the distribution of particles in phase space — the space of positions $\{x, y, z\}$ and momenta $\{p_x, p_y, p_z\}$. (Here $z$ points along the direction of motion and $x$ and $y$ are in a plane transverse to $z$. We'll ignore the $z$ dimension for now.)

There are a few methods to measure the phase space distribution of a beam. The most direct way is to use one slit to block all particles falling outside of a narrow horizontal region $x \pm \Delta_{x}$, a second slit to select a narrow range of momentum $p_x \pm \Delta_{p_x}$, and so on for $y$ and $yp$. This takes a long time and is invasive because most of the beam is blocked. There are also indirect methods in which the covariance matrix of the distribution is reconstructed multiple 1D projections; these are fast and non-invasive, and in many cases they provide enough information. Finally, there is the in-between method of tomographic reconstruction.

In my current project, we're trying to create a beam in the Spallation Neutron Source (SNS) that has an elliptical shape in the $x$-$y$ plane, a uniform charge density, and a large angular momentum. We'd like to measure the four-dimensional (4D) phase space distribution of this beam, which is composed of approximately $10^{14}$ protons moving at 90% the speed of light. 

A slit-scan is not possible at this energy. The SNS has a number of wire-scanners — devices to measure 1D projections of the beam by sweeping a wire across the beam path. By measuring the projections at different angles and varying the magnet strenghs between the wire-scanners, it's possible to reconstruct the beam covariance matrix. I've implemented this method and used it in recent experiments.

The SNS also has a target imaging system. The Mercury target at the very end of the accelerator is coated with a luminescent material that glows when impacted by the proton beam. This allows us to view an image of the beam on the target. It's recently been shown that a 4D distribution can be reconstructed from multiple 2D projections of the beam on a screen under certain conditions, and there are more general methods that can reconstruct the distribution with arbitrary view angles. 

## Reconstruct 2D phase space distribution 

Create the distribution.

In [ ]:
X = np.random.multivariate_normal(mean=[0., 0.], cov=[[1.5, 0.5], [0.5, 1.0]], size=500000)
limit = (-4.5, 4.5)
bins = 50
Z, xedges, yedges = np.histogram2d(X[:, 0], X[:, 1], bins, (limit, limit))
Z /= np.sum(Z)
xcenters = 0.5 * (xedges[:-1] + xedges[1:])
ycenters = 0.5 * (yedges[:-1] + yedges[1:])
xgrid, ygrid = np.meshgrid(xcenters, ycenters)

fig, ax = pplt.subplots()
ax.pcolormesh(xgrid, ygrid, Z.T, cmap='dusk_r')
ax.format(xlabel='x', ylabel='y')
plt.show()

Simulate the measurements. For now we assume the transfer matrices are rotation matrices.

In [ ]:
n_projections = 50
angles = np.linspace(0., np.pi, n_projections, endpoint=False)
projections = np.zeros((bins, len(angles)))
for i, angle in enumerate(tqdm(angles)):
    M = utils.rotation_matrix(angle)
    Xrot = np.apply_along_axis(lambda row: np.matmul(M, row), 1, X)
    projection, _ = np.histogram(Xrot[:, 0], bins, limit)
    projections[:, i] = projection

### FBP

Notice that our matrix `Z` has $x$ along the first axis and $y$ along the second axis, so when we display this image using `plt.pcolormesh`, we display `Z.T`. But `skimage` thinks the projections are of `Z.T`.

In [ ]:
skimage_angles = 90.0 + np.degrees(angles)

In [ ]:
Z_rec = iradon(projections, theta=skimage_angles)
Z_rec /= np.sum(Z_rec)

In [ ]:
fig, axes = pplt.subplots(ncols=3, figsize=None)
kws = dict(cmap='dusk_r', shading='auto')
axes[0].pcolormesh(xgrid, ygrid, Z.T, **kws)
axes[1].pcolormesh(xgrid, ygrid, Z_rec.T, **kws)
axes[2].pcolormesh(xgrid, ygrid, (Z - Z_rec).T, cmap='RdBu', colorbar=True)
axes.format(xlabel="x", ylabel="x'")
axes[0].set_title('True')
axes[1].set_title('Reconstructed');
axes[2].set_title('abs(Error)');

### MENT

In [ ]:
skip = 10
projections_ = projections[:, ::skip]
angles_ = angles[::skip]

In [ ]:
proj_ = radon(Z.T, theta=np.degrees(angles_))
fig, ax = pplt.subplots()
ax.pcolormesh(xgrid, ygrid, iradon(proj_, theta=np.degrees(angles_)), cmap='dusk_r')
plt.show()

## 4D phase space reconstruction 

In [ ]:
# Create a 4D Gaussian distribution.
X = np.random.normal(scale=[1., 1., 1., 1.], size=(100000, 4))

# Add cross-plane correlations.
a, b, c = 0.3, 0.1, -0.4
d = b * c / a if a != 0. else 0.
V = np.array([[1., 0., a, b],
              [0., 1., c, d],
              [-d, b, 1., 0],
              [c, -a, 0., 1.]])
X = utils.apply(V, X)

# Plot distribution and save axis limits for later use.
axes = myplt.corner(X, pad=-0.1, bins=bins)
limits = [ax.get_xlim() for ax in axes[-1, :]]

Simulate the measurements. $S_{ijkl}$ gives the density on the $i$,$j$ screen pixel for the $k$th x angle and $l$th y angle.

In [ ]:
def transfer_matrix(mux, muy):
    M = np.zeros((4, 4))
    M[:2, :2] = utils.rotation_matrix(mux)
    M[2:, 2:] = utils.rotation_matrix(muy)
    return M

n_projections = 50
angles = np.linspace(0., np.pi, n_projections, endpoint=False)
xx_list, yy_list = [], [] 
for angle in tqdm(angles):
    M = transfer_matrix(angle, angle)
    x, xp, y, yp = utils.apply(M, X).T
    xx_list.append(x)
    yy_list.append(y)

In [ ]:
S = np.zeros((bins, bins, len(angles), len(angles)))
for k in trange(len(angles)):
    for l in range(len(angles)):
        xx = xx_list[k]
        yy = yy_list[l]
        H, _, _ = np.histogram2d(xx, yy, bins, (limits[0], limits[2]))
        S[:, :, k, l] = H

We first reconstruct the 3D distribution — $x$, $x'$, $y$. Consider a fixed $y$, meaning one row of the beam image. The intensity along that row gives a 1D projection onto the $x$ plane for this vertical slice. But we have a bunch of these 1D projections at different angles in the $x$-$x'$ plane at the reconstruction location; we can use them to reconstruct the $x$-$x'$ distribution at this vertical slice, and repeat for each slice. Thus, we obtain an array $D$ such that $D_{jlrs}$ gives the density at $x = x_r$, $x' = x'_s$ for $y = y_j$ and $\theta_y = \pi l / K$. 

In [ ]:
grid_size = 50 # Use the same grid in the reconstruction
D = np.zeros((bins, len(angles), grid_size, grid_size))
for j in trange(bins):
    for l in range(len(angles)):
        projections = S[:, j, :, l]
        D[j, l, :, :] = iradon(projections, theta=-np.degrees(angles), output_size=grid_size)

We can do a similar thing in the other plane. Now we chose one cell in the $x$-$x'$ grid; at this cell, there is a range of $y$ values giving a 1D projection onto the $y$ axis. And we have a set of these 1D projections, each of which is at a different angle in $y$-$y'$ at the reconstruction location. Thus, we obtain an array $F$ such that $F_{rstu}$ gives the density at $x = x_r$, $x' = x'_s$ for $y = y_s$ and $y'=y'_u$, i.e., we have the 4D phase space density. 

In [ ]:
F = np.zeros((grid_size, grid_size, grid_size, grid_size))
for r in trange(grid_size):
    for s in range(grid_size):
        projections = D[:, :, r, s]
        F[r, s, :, :] = iradon(projections, theta=-np.degrees(angles), output_size=grid_size)

The true distribution can be obtained from a 4D histogram of the original data.

In [ ]:
F_true, edges = np.histogramdd(X, grid_size, limits)

In [ ]:
def project(F, indices):
    axis = tuple([k for k in range(4) if k not in indices])
    return np.sum(F, axis=axis)

def get_projections(F):
    projections = [[], [], []]
    for i in range(3):
        for j in range(i + 1):
            projection = project(F, [j, i + 1])
            projections[i].append(projection)
    return projections

def plot_projections(projections, limits):
    pcolormesh_kws = dict(cmap='dusk_r', shading='auto')
    labels = ['x', 'xp', 'y', 'yp']
    fig, axes = myplt.pair_grid_nodiag(4, limits=limits)
    for i in range(3):
        for j in range(i + 1):
            ax = axes[i, j]
            Z = projections[i][j]
            xlim, ylim = ax.get_xlim(), ax.get_ylim()
            xx = np.linspace(xlim[0], xlim[1], Z.shape[0])
            yy = np.linspace(ylim[0], ylim[1], Z.shape[1])
            ax.pcolormesh(xx, yy, Z.T, **pcolormesh_kws)
    return axes

projections_true = get_projections(F)
axes = plot_projections(projections_true, limits)
plt.suptitle('Original')

projections = get_projections(F_true)
axes = plot_projections(projections, limits)
plt.suptitle('Reconstructed')

projections_err = get_projections(np.abs(F - F_true))
axes = plot_projections(projections_err, limits)
plt.suptitle('abs(error)');